### Automated Shiptrack detection model hyperparameter tuning

Train and deploy the shiptrack detection model using the built-in Tensorflow containers provided by Amazon SageMaker.

This is heavily based on the keras-05-keras-blog-post_Fashion MNIST-SageMaker example.  

In [ ]:
import sagemaker

sess = sagemaker.Session()
role = sagemaker.get_execution_role()

## Train with Tensorflow on a GPU instance

In [ ]:
from sagemaker.tensorflow import TensorFlow

In [ ]:
training_input_path = 's3://imiracli-data/MODIS_deep_cloud/compressed_training/nocrop_combined_points'

tf_estimator = TensorFlow(entry_point='shiptrack.py', 
                          role=role,
                          train_instance_count=1, 
                          train_instance_type='ml.p3.2xlarge',
                          framework_version='1.12', 
                          py_version='py3',
                          source_dir = './shiptrack-detection/',
                          script_mode=True,
                          hyperparameters={
                              'epochs': 30,
                              'batch-size': 8,
                              'learning-rate': 0.01}
                         )

In [ ]:
tf_estimator.fit(training_input_path)

## Deploy

In [ ]:
import time

tf_endpoint_name = 'shiptrack-detection-'+time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

#tf_predictor = tf_estimator.deploy(initial_instance_count=1,
#                                   instance_type='ml.p2.xlarge')      # $1.361/hour in eu-west-1

tf_predictor = tf_estimator.deploy(initial_instance_count=1,
                         instance_type='ml.c5.large',        # $0.134/hour in eu-west-1
                         accelerator_type='ml.eia1.medium',  # + $0.140/hour in eu-west-1
                         endpoint_name=tf_endpoint_name)     # = 80% discount!

## Predict 

In [ ]:
%matplotlib inline
import random
import matplotlib.pyplot as plt

num_samples = 5
indices = random.sample(range(x_val.shape[0] - 1), num_samples)
images = x_val[indices]/255
labels = y_val[indices]

for i in range(num_samples):
    plt.subplot(1,num_samples,i+1)
    plt.imshow(images[i].reshape(28, 28), cmap='gray')
    plt.title(labels[i])
    plt.axis('off')
    
prediction = tf_predictor.predict(images.reshape(num_samples, 28, 28, 1))['predictions']
prediction = np.array(prediction)
predicted_label = prediction.argmax(axis=1)
print('Predicted labels are: {}'.format(predicted_label))

## Clean up

In [ ]:
sess.delete_endpoint(endpoint_name=tf_endpoint_name)

## Configure Automatic Model Tuning

In [ ]:
tf_estimator = TensorFlow(entry_point='shiptrack.py', 
                          role=role,
                          train_instance_count=1, 
                          train_instance_type='ml.p3.2xlarge',
                          framework_version='1.12', 
                          py_version='py3',
                          script_mode=True
                         )

In [ ]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

hyperparameter_ranges = {
    'epochs':        IntegerParameter(20, 100),
    'learning-rate': ContinuousParameter(0.001, 0.1, scaling_type='Logarithmic'), 
    'batch-size':    IntegerParameter(32, 1024),
    'dense-layer':   IntegerParameter(128, 1024),
    'dropout':       ContinuousParameter(0.2, 0.6)
}

objective_metric_name = 'val_acc'
objective_type = 'Maximize'
metric_definitions = [{'Name': 'val_acc', 'Regex': 'val_acc: ([0-9\\.]+)'}]

tuner = HyperparameterTuner(tf_estimator,
                            objective_metric_name,
                            hyperparameter_ranges,
                            metric_definitions,
                            max_jobs=10,
                            max_parallel_jobs=2,
                            objective_type=objective_type)

In [ ]:
tuner.fit(training_input_path)

## Deploy the best model

In [ ]:
import time

tf_endpoint_name = 'shiptrack-detection-'+time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

tf_predictor = tuner.deploy(initial_instance_count=1,
                         instance_type='ml.c5.large',       
                         accelerator_type='ml.eia1.medium',
                         endpoint_name=tf_endpoint_name)

## Clean up

In [ ]:
sess.delete_endpoint(endpoint_name=tf_endpoint_name)